In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df=pd.read_csv("data/odb.csv")

In [3]:
df.head()

,Unnamed: 0,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s
0,0,SR Tendulkar (INDIA),1989-2012,463,452,41,18426,200*,44.83,21368,86.23,49,96,20,2016,195
1,1,V Kohli (INDIA),2008-2021,254,245,39,12169,183,59.07,13061,93.17,43,62,13,1140,125
2,2,RT Ponting (AUS/ICC),1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82,20,1231,162
3,3,RG Sharma (INDIA),2007-2021,227,220,32,9205,264,48.96,10354,88.90,29,43,13,832,244
4,4,ST Jayasuriya (Asia/SL),1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68,34,1500,270


In [4]:
df.dtypes

Unnamed: 0      int64
Player         object
Span           object
Mat             int64
Inns            int64
NO              int64
Runs            int64
HS             object
Ave           float64
BF              int64
SR            float64
100             int64
50              int64
0               int64
4s             object
6s             object
dtype: object

In [5]:
df.isnull().sum()

Unnamed: 0    0
Player        0
Span          0
Mat           0
Inns          0
NO            0
Runs          0
HS            0
Ave           0
BF            0
SR            0
100           0
50            0
0             0
4s            0
6s            0
dtype: int64

In [6]:
df.drop(['Span','NO','Unnamed: 0'],axis=1,inplace=True)

In [7]:
df.columns

Index(['Player', 'Mat', 'Inns', 'Runs', 'HS', 'Ave', 'BF', 'SR', '100', '50',
       '0', '4s', '6s'],
      dtype='object')

In [8]:
df['HS'] = df['HS'].replace(r"\*", "", regex=True)

In [9]:
df.dtypes

Player     object
Mat         int64
Inns        int64
Runs        int64
HS         object
Ave       float64
BF          int64
SR        float64
100         int64
50          int64
0           int64
4s         object
6s         object
dtype: object

In [10]:
df['HS'] = pd.to_numeric(df['HS'], errors='coerce')
# Clean the 'HS' column in the EDA phase by removing non-numeric characters and converting to float
df['HS'] = df['HS'].replace(to_replace=r'\D', value='', regex=True).astype(float)

In [11]:
# Clean the 'HS' column in the data by removing non-numeric characters and converting to float
df['HS'] = df['HS'].replace(to_replace=r'\D', value='', regex=True)  # Remove non-numeric characters
df['HS'] = pd.to_numeric(df['HS'], errors='coerce')  # Convert to numeric, invalid parsing will be NaN

In [12]:
df['4s'] = pd.to_numeric(df['4s'], errors='coerce')
df['6s'] = pd.to_numeric(df['6s'], errors='coerce')

In [13]:
df.dtypes

Player     object
Mat         int64
Inns        int64
Runs        int64
HS        float64
Ave       float64
BF          int64
SR        float64
100         int64
50          int64
0           int64
4s        float64
6s        float64
dtype: object

In [ ]:

df['Player']=df['Player']